# Q2 — Repeat Offenders & Interarrival (Exact Recompute)
**Source:** `violations_routes_filtered.csv`
**Scope:** Routes M101, M60+, M15+; `is_exempt == True`
**Outputs match prior chat deliverables.**

In [ ]:
# imports
import pandas as pd, numpy as np
from pathlib import Path
from IPython.display import display  # for displaying DataFrames in notebook


In [ ]:
# load data and basic setup
DATA = "/mnt/data/violations_routes_filtered.csv"
print("Reading:", DATA)
usecols = ["Bus Route ID","Datetime","is_exempt","Violation Status","Stop Name","Vehicle ID"]
df = pd.read_csv(DATA, usecols=usecols, low_memory=True)
df.columns = [c.strip() for c in df.columns]


In [ ]:
# tag routes of interest (normalize M101, M60+, M15+)
def route_tag(r):
    if pd.isna(r): return None
    r = str(r).upper().strip()
    if r.startswith("M101"): return "M101"
    if r.startswith("M60"):  return "M60+" if ("SBS" in r or "+" in r or "-SBS" in r or "-" in r) else "M60"
    if r.startswith("M15"):  return "M15+" if (("SBS" in r) or ("+" in r) or ("-SBS" in r)) else "M15"
    return r

df["route_tag"] = df["Bus Route ID"].apply(route_tag)
df = df[df["route_tag"].isin(["M101","M60+","M15+"])]
df["is_exempt"] = df["is_exempt"].astype(str).str.lower().isin(["true","1","t","yes","y"])


In [ ]:
# create exempt only frame and parse dates
ex = df[df["is_exempt"]==True].copy()
ex["Datetime"] = pd.to_datetime(ex["Datetime"], errors="coerce", infer_datetime_format=True)


In [ ]:
# helpers to build repeat-offender buckets and top offenders per route
def bucket_table(route):
    d = ex[(ex["route_tag"]==route) & (ex["Vehicle ID"].notna())]
    vc = d.groupby("Vehicle ID").size()
    bins = [0,1,5,10,25,50,10**9]
    labels = ["=1","2-5","6-10","11-25","26-50",">50"]
    cats = pd.cut(vc, bins=bins, labels=labels, right=True, include_lowest=True)
    dist = cats.value_counts().reindex(labels).fillna(0).astype(int)
    dist_pct = (dist/dist.sum()*100).round(2)
    out = pd.DataFrame({"count": dist, "pct": dist_pct})
    out.loc["unique_exempt_vehicles"] = [int(vc.shape[0]), 100.0]
    return out, vc.sort_values(ascending=False)


In [ ]:
# build summaries and show top repeat offenders per route
summary = {}
tops = {}
for r in ["M101","M60+","M15+"]:
    tab, vc = bucket_table(r)
    summary[r] = tab
    tops[r] = vc.head(10)

for r in summary:
    print(f"\n=== {r} — Violations per Vehicle (counts & %) ===")
    display(summary[r])
    print(f"\n{r} — Top 10 offenders (counts)")
    display(tops[r])


In [ ]:
# compute inter-arrival (days between repeats) and summarize by route
e = ex.dropna(subset=["Vehicle ID","Datetime"]).sort_values(["Vehicle ID","Datetime"]).copy()
e["prev_vid"] = e["Vehicle ID"].shift(1); e["prev_time"] = e["Datetime"].shift(1)
same = e["Vehicle ID"]==e["prev_vid"]
e["delta_days"] = np.where(same, (e["Datetime"]-e["prev_time"]).dt.total_seconds()/(3600*24), np.nan)
intervals = e.loc[same & e["delta_days"].notna(), ["route_tag","Vehicle ID","delta_days"]]

inter = (intervals.groupby("route_tag")["delta_days"]
         .agg(n_intervals="count",
              median_days_between=lambda s: round(float(s.median()),2),
              mean_days_between=lambda s: round(float(s.mean()),2),
              p75_days_between=lambda s: round(float(s.quantile(0.75)),2),
              p90_days_between=lambda s: round(float(s.quantile(0.90)),2))
         .reset_index())

rep_counts = intervals.groupby("route_tag")["Vehicle ID"].nunique().rename("n_repeat_vehicles").reset_index()
inter = inter.merge(rep_counts, on="route_tag", how="left")
display(inter)


In [ ]:
# save csv outputs for distributions, top offenders, and inter-arrival summary
outdir = Path("/mnt/data/q2_notebooks/outputs"); outdir.mkdir(parents=True, exist_ok=True)
for r, tab in summary.items():
    tab.to_csv(outdir/f"repeat_distribution_{r}.csv")
for r, vc in tops.items():
    vc.to_csv(outdir/f"top10_offenders_{r}.csv", header=["exempt_violations_count"])
inter.to_csv(outdir/"interarrival_summary.csv", index=False)
print("Saved outputs to", outdir)
